In [2]:
import cx_Oracle
import pandas as pd
import datetime

def format_value(value):
    if value is None:
        return 'NULL'
    elif isinstance(value, str):
        return f"'{value}'"
    elif isinstance(value, datetime.date):
        formatted_date = value.strftime("%Y-%m-%d %H:%M:%S")
        return f"TO_DATE('{formatted_date}', 'YYYY-MM-DD HH24:MI:SS')"
    else:
        return str(value)

def get_next_sequence_value(target_cursor, sequence_name):
    # Function to get the next value from a sequence
    target_cursor.execute(f"SELECT {sequence_name}.NEXTVAL FROM dual")
    return target_cursor.fetchone()[0]

def val_map_insert(vm_df, target_vm_table, target_cursor, target_conn, sequence_name, map_definition_id, OVERRIDE_FLAG, old_map_def_id):
    try:
        # Remove the first column from vm_df
        vm_df = vm_df.iloc[:, 2:]
        
        # Construct the INSERT INTO statement
        vm_column_names = vm_df.columns.tolist()
        
        insert_vm = f"""
            INSERT INTO {target_vm_table} (
                MAP_ID, MAP_DEFINITION_ID, {', '.join(vm_column_names)}  
            ) VALUES (
                :MAP_ID, :MAP_DEFINITION_ID, :{', :'.join(vm_column_names)} 
            )
        """
        
        for index, row in vm_df.iterrows():
            # Get the next sequence value
            vm_values = {'MAP_ID': get_next_sequence_value(target_cursor, sequence_name)}
            vm_values['MAP_DEFINITION_ID'] = map_definition_id  # Use the provided map_definition_id
            
            # Populate the values dictionary from the DataFrame row
            for col in vm_column_names:
                vm_values[col] = row[col]

            if OVERRIDE_FLAG:
                # Delete existing records with the same MAP_DEFINITION_ID
                delete_vm_query = f"""DELETE FROM {target_vm_table} WHERE MAP_DEFINITION_ID = {old_map_def_id}"""
                target_cursor.execute(delete_vm_query)
                target_conn.commit()
                print(f"Deleted existing records for MAP_DEFINITION_ID: {old_map_def_id}")

            # Execute the INSERT statement with the values dictionary
            target_cursor.execute(insert_vm, vm_values)
            target_conn.commit()  # Commit the transaction
            print(f"Row {index} inserted successfully.")
            
    except cx_Oracle.Error as error:
        print(f"Error executing VM INSERT statement: {error}")


In [42]:
import cx_Oracle
import pandas as pd
import time

# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)

# Target database connection
target_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)


start_time = time.time()
print(f"Start of Task: at {start_time}")

OVERRIDE_FLAG = True
p_map_code = "OTC_ORDER_ENTRY_UI"

# Create cursors for both connections
source_cursor = source_conn.cursor()
target_cursor = target_conn.cursor()

# Select data from the source table
sql_query = f"""SELECT * FROM APPS.XXCMN_VALUE_MAP_DEF WHERE MAP_CODE = '{p_map_code}'"""
source_cursor.execute(sql_query)

# Fetch the data and the column names
def_df = source_cursor.fetchall()


# Retrieve the column names from the cursor description
def_column_names = [desc[0] for desc in source_cursor.description]

# Create a DataFrame and set the column names
def_df = pd.DataFrame(def_df, columns=def_column_names)
print(def_df)
column_types = def_df.dtypes
for i in column_types:
# Print the data types
    print('Print the data types',i)

# Now you can reference columns by their names
pn_map_def_id = int(def_df['MAP_DEFINITION_ID'].iloc[0])
print('pn_map_def_id :',pn_map_def_id)

vm_query = f"""SELECT *
                 FROM APPS.XXCMN_VALUE_MAP 
                 WHERE MAP_DEFINITION_ID = {pn_map_def_id}
                 ORDER BY map_id"""
source_cursor.execute(vm_query)

# Fetch the data and the column names
vm_df = source_cursor.fetchall()

# Retrieve the column names from the cursor description
vm_column_names = [desc[0] for desc in source_cursor.description]

# Create a DataFrame and set the column names
vm_df = pd.DataFrame(vm_df, columns=vm_column_names)
print(vm_df)

# Close the source cursor and connection
source_cursor.close()
source_conn.close()

# Construct the INSERT INTO statement for DEF

def_df = def_df.iloc[:, 1:]

# Specify the target table name
target_table = "OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST"

# Construct the INSERT INTO statement
insert_def = f"""
    INSERT INTO {target_table} (
        MAP_DEFINITION_ID, {', '.join(def_column_names[1:])}  
    ) VALUES (
        :MAP_DEFINITION_ID ,:{', :'.join(def_column_names[1:])} 
    )
"""

print('pn_map_def_id :',pn_map_def_id)

# Create a cursor for the target database connection
target_cursor = target_conn.cursor()

# Initialize the values dictionary
target_cursor.execute("SELECT OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST_S.NEXTVAL FROM dual")
next_val = target_cursor.fetchone()[0]
values = {'MAP_DEFINITION_ID': next_val}  # Start with the variable value

# Populate the values dictionary from the DataFrame
for col in def_column_names[1:]:
    values[col] = def_df[col].iloc[0]  # Get values from the DataFrame

# Execute the INSERT statement with the values dictionary
try:
    # Execute your SQL query to check if the MAP_CODE already exists
    target_cursor.execute(f"SELECT MAP_DEFINITION_ID,MAP_CODE FROM {target_table} WHERE MAP_CODE = :p_map_code", {"p_map_code": p_map_code})
#     result = target_cursor.fetchone()

    # Fetch the results
    result = target_cursor.fetchone()
    old_map_def_id =result[0]
    print(old_map_def_id)
    if result:
        existing_map_code = result[1]
        print(f"MAP_CODE already exists with Existing MAP_CODE: {existing_map_code}")
        
        if OVERRIDE_FLAG:  # Check if OVERRIDE_FLAG is True
            # Delete existing records with the same MAP_CODE
            delete_query = f"DELETE FROM {target_table} WHERE MAP_CODE = :p_map_code"
            print(delete_query)
            target_cursor.execute(delete_query, {"p_map_code": p_map_code})
            target_conn.commit()
            
            # Insert the new records
            print('insert_print',insert_print)
            insert_print = target_cursor.execute(insert_def, values)
            print(insert_print)
            target_conn.commit()
            print(f"Deleted existing records and inserted new data for MAP_CODE: {p_map_code}")
            print(f"Inserted a new row with MAP_DEFINITION_ID: {next_val}")
            print('pn_map_def_id befoe calling:',old_map_def_id)
            val_map_insert(vm_df, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST", target_cursor, target_conn, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S", next_val,OVERRIDE_FLAG,old_map_def_id)
        
    else:
        # Execute the INSERT statement with the values dictionary
        print(insert_def)
        target_cursor.execute(insert_def, values)
        target_conn.commit()  # Commit the transaction
        print("INSERT statement executed successfully.")
        print(f"Inserted a new row with MAP_DEFINITION_ID: {next_val}")
        
        # Call the val_map_insert function for value map data
        val_map_insert(vm_df, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST", target_cursor, target_conn, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S", next_val,OVERRIDE_FLAG,old_map_def_id)

except cx_Oracle.Error as error:
    print(f"Error executing DEF INSERT statement: {error}")

# Close the target cursor and connection
# target_cursor.close()
# target_conn.close()


Start of Task: at 1697611034.2831454
   MAP_DEFINITION_ID            MAP_CODE                           MAP_DESC  \
0                301  OTC_ORDER_ENTRY_UI  OTC Order Entry UI List of Values   

  ACTIVE_DATE_FLAG N_VALUE1_SEQUENCE N_VALUE1_PROMPT_TEXT  \
0                Y              None                 None   

  N_VALUE1_REQUIRED_FLAG N_VALUE1_COLUMN_WIDTH C_VALUE1_SEQUENCE  \
0                   None                  None              None   

  C_VALUE1_PROMPT_TEXT  ... C_VALUE19_UI_SHOW C_VALUE19_UI_UPDATE  \
0             Division  ...              None                None   

  D_VALUE19_UI_SHOW D_VALUE19_UI_UPDATE N_VALUE20_UI_SHOW N_VALUE20_UI_UPDATE  \
0              None                None              None                None   

  C_VALUE20_UI_SHOW C_VALUE20_UI_UPDATE D_VALUE20_UI_SHOW D_VALUE20_UI_UPDATE  
0              None                None              None                None  

[1 rows x 375 columns]
Print the data types int64
Print the data types object
Pri

   MAP_ID  MAP_DEFINITION_ID N_VALUE1 C_VALUE1 D_VALUE1 N_VALUE2   C_VALUE2  \
0    7464                301     None      AWI     None     None  Immediate   
1    7466                301     None      AWI     None     None       None   
2   26269                301     None       GS     None     None  Immediate   
3   26270                301     None       GS     None     None       None   

  D_VALUE2 N_VALUE3         C_VALUE3  ... N_VALUE20 C_VALUE20 D_VALUE20  \
0     None     None  Customer Pickup  ...      None      None      None   
1     None     None            Truck  ...      None      None      None   
2     None     None  Customer Pickup  ...      None      None      None   
3     None     None            Truck  ...      None      None      None   

  ACTIVE_BEGIN_DATE ACTIVE_END_DATE CREATED_BY       CREATION_DATE  \
0              None            None     1088.0 2021-08-03 09:46:28   
1              None            None     1088.0 2021-08-03 09:48:20   
2              Non

TypeError: 'NoneType' object is not subscriptable

In [53]:
# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)
source_cursor = source_conn.cursor()

In [54]:
table_name = 'XXCMN_VALUE_MAP_DEF_TEST'

# Create a cursor

# Execute a SQL query to retrieve column information
query = f"""
SELECT COLUMN_NAME, DATA_TYPE
FROM ALL_TAB_COLUMNS
WHERE TABLE_NAME = '{table_name}'
"""
source_cursor.execute(query)

# Fetch the results
column_information = source_cursor.fetchall()

# Create empty lists to store column names for different data types
date_columns = []
number_columns = []
varchar_columns = []

# Iterate through the column information
for column_info in column_information:
    column_name, data_type = column_info
    if data_type == 'DATE':
        date_columns.append(column_name)
    elif data_type in ('NUMBER', 'FLOAT', 'INT'):
        number_columns.append(column_name)
    elif data_type in ('VARCHAR2', 'CHAR', 'VARCHAR'):
        varchar_columns.append(column_name)

# Print the sorted lists
print("Date Columns:", date_columns)
print("Number Columns:", number_columns)
print("Varchar Columns:", varchar_columns)

# Close the cursor
source_cursor.close()

# Close the connection
source_conn.close()

# Display the column descriptions
for desc in column_information:
    print(desc)

Date Columns: ['ACTIVE_BEGIN_DATE', 'ACTIVE_END_DATE', 'CREATION_DATE', 'LAST_UPDATE_DATE']
Number Columns: ['MAP_DEFINITION_ID', 'N_VALUE1_SEQUENCE', 'N_VALUE1_COLUMN_WIDTH', 'C_VALUE1_SEQUENCE', 'C_VALUE1_COLUMN_WIDTH', 'D_VALUE1_SEQUENCE', 'D_VALUE1_COLUMN_WIDTH', 'N_VALUE2_SEQUENCE', 'N_VALUE2_COLUMN_WIDTH', 'C_VALUE2_SEQUENCE', 'C_VALUE2_COLUMN_WIDTH', 'D_VALUE2_SEQUENCE', 'D_VALUE2_COLUMN_WIDTH', 'N_VALUE3_SEQUENCE', 'N_VALUE3_COLUMN_WIDTH', 'C_VALUE3_SEQUENCE', 'C_VALUE3_COLUMN_WIDTH', 'D_VALUE3_SEQUENCE', 'D_VALUE3_COLUMN_WIDTH', 'N_VALUE4_SEQUENCE', 'N_VALUE4_COLUMN_WIDTH', 'C_VALUE4_SEQUENCE', 'C_VALUE4_COLUMN_WIDTH', 'D_VALUE4_SEQUENCE', 'D_VALUE4_COLUMN_WIDTH', 'N_VALUE5_SEQUENCE', 'N_VALUE5_COLUMN_WIDTH', 'C_VALUE5_SEQUENCE', 'C_VALUE5_COLUMN_WIDTH', 'D_VALUE5_SEQUENCE', 'D_VALUE5_COLUMN_WIDTH', 'N_VALUE6_SEQUENCE', 'N_VALUE6_COLUMN_WIDTH', 'C_VALUE6_SEQUENCE', 'C_VALUE6_COLUMN_WIDTH', 'D_VALUE6_SEQUENCE', 'D_VALUE6_COLUMN_WIDTH', 'N_VALUE7_SEQUENCE', 'N_VALUE7_COLUMN_WIDT

('MAP_DEFINITION_ID', 'NUMBER')
('MAP_CODE', 'VARCHAR2')
('MAP_DESC', 'VARCHAR2')
('ACTIVE_DATE_FLAG', 'VARCHAR2')
('N_VALUE1_SEQUENCE', 'NUMBER')
('N_VALUE1_PROMPT_TEXT', 'VARCHAR2')
('N_VALUE1_REQUIRED_FLAG', 'VARCHAR2')
('N_VALUE1_COLUMN_WIDTH', 'NUMBER')
('C_VALUE1_SEQUENCE', 'NUMBER')
('C_VALUE1_PROMPT_TEXT', 'VARCHAR2')
('C_VALUE1_REQUIRED_FLAG', 'VARCHAR2')
('C_VALUE1_COLUMN_WIDTH', 'NUMBER')
('D_VALUE1_SEQUENCE', 'NUMBER')
('D_VALUE1_PROMPT_TEXT', 'VARCHAR2')
('D_VALUE1_REQUIRED_FLAG', 'VARCHAR2')
('D_VALUE1_COLUMN_WIDTH', 'NUMBER')
('N_VALUE2_SEQUENCE', 'NUMBER')
('N_VALUE2_PROMPT_TEXT', 'VARCHAR2')
('N_VALUE2_REQUIRED_FLAG', 'VARCHAR2')
('N_VALUE2_COLUMN_WIDTH', 'NUMBER')
('C_VALUE2_SEQUENCE', 'NUMBER')
('C_VALUE2_PROMPT_TEXT', 'VARCHAR2')
('C_VALUE2_REQUIRED_FLAG', 'VARCHAR2')
('C_VALUE2_COLUMN_WIDTH', 'NUMBER')
('D_VALUE2_SEQUENCE', 'NUMBER')
('D_VALUE2_PROMPT_TEXT', 'VARCHAR2')
('D_VALUE2_REQUIRED_FLAG', 'VARCHAR2')
('D_VALUE2_COLUMN_WIDTH', 'NUMBER')
('N_VALUE3_SEQUENCE',

In [15]:
# Target database connection
target_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)
target_cursor = target_conn.cursor()

target_cursor.execute(f"SELECT MAP_DEFINITION_ID,MAP_CODE FROM {target_table} WHERE MAP_CODE = :p_map_code", {"p_map_code": p_map_code})
result = target_cursor.fetchone()
old_map_def_id =result[0]
print(old_map_def_id)
if result:
    existing_map_code = result[1]
    print(f"MAP_CODE already exists with Existing MAP_CODE: {existing_map_code}")

163
MAP_CODE already exists with Existing MAP_CODE: OTC_ORDER_ENTRY_UI


In [1]:
sql_file = open('execute.sql', 'w')


In [4]:
# Log the INSERT statement to the file
sql_file.write(insert_vm + ';\n')  # Add a semicolon at the end
sql_file.write(f"/* Values: {str(vm_values)} */\n")


NameError: name 'insert_vm' is not defined

In [5]:
print('Hello')

Hello


In [15]:
# Define pn_map_def_id somewhere in your code
pn_map_def_id = 12345  # Replace with the actual value

# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)
source_cursor = source_conn.cursor()

# Construct the SQL query
vm_query = f"""SELECT *
                 FROM APPS.XXCMN_VALUE_MAP 
                 WHERE MAP_DEFINITION_ID = {pn_map_def_id}
                 ORDER BY map_id"""

# Print the query and the value of pn_map_def_id to the console
print("Query being executed:")
print(vm_query)
print(f"pn_map_def_id: {pn_map_def_id}")

# Write the query to a select.sql file
with open('select.sql', 'w') as sql_file:
    sql_file.write(vm_query)

# Execute the query with source_cursor
a = source_cursor.execute(vm_query)
print(a)


Query being executed:
SELECT *
                 FROM APPS.XXCMN_VALUE_MAP 
                 WHERE MAP_DEFINITION_ID = 12345
                 ORDER BY map_id
pn_map_def_id: 12345
<cx_Oracle.Cursor on <cx_Oracle.Connection to apps@omsddb.cswg.com:1521/omsd_app>>


In [25]:
# Define pn_map_def_id and p_map_code somewhere in your code
pn_map_def_id = 12345  # Replace with the actual value
p_map_code = "your_map_code"  # Replace with the actual value
target_table = "your_target_table"  # Replace with the actual table name

# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)
source_cursor = source_conn.cursor()

# Construct the first SQL query
vm_query = f"""SELECT *
                 FROM APPS.XXCMN_VALUE_MAP 
                 WHERE MAP_DEFINITION_ID = {pn_map_def_id}
                 ORDER BY map_id"""

# Append a slash on the next line
vm_query += ";\n"

# Print the query and the value of pn_map_def_id to the console
print("First Query being executed:")
print(vm_query)
print(f"pn_map_def_id: {pn_map_def_id}")

# Write the first query to a select.sql file
with open('select.sql', 'w') as sql_file:
    sql_file.write(vm_query)
    sql_file.write('/\n')
    
# with open('select.sql', 'a') as sql_file:
#     sql_file.write('/\n')
    

# Execute the first query with source_cursor
# a = source_cursor.execute(vm_query)
# print(a)

# Target database connection
# target_conn = cx_Oracle.connect(
#     user="target_user",
#     password="target_password",
#     dsn="target_db_connection"
# )
# target_cursor = target_conn.cursor()

# Construct the second SQL query
target_query = f"SELECT MAP_DEFINITION_ID, MAP_CODE FROM {target_table} WHERE MAP_CODE = :p_map_code"

# Append a slash on the next line
target_query += ";\n"

# Print the query and the value of p_map_code to the console
print("Second Query being executed:")
print(target_query)
print(f"p_map_code: {p_map_code}")

# Write the second query to the select.sql file (append mode)
with open('select.sql', 'a') as sql_file:
    sql_file.write(target_query)
#     sql_file.write(';')
    sql_file.write('/\n')

# Execute the second query with target_cursor
# result = target_cursor.execute(target_query, {"p_map_code": p_map_code})
# print(result.fetchone())


First Query being executed:
SELECT *
                 FROM APPS.XXCMN_VALUE_MAP 
                 WHERE MAP_DEFINITION_ID = 12345
                 ORDER BY map_id;

pn_map_def_id: 12345
Second Query being executed:
SELECT MAP_DEFINITION_ID, MAP_CODE FROM your_target_table WHERE MAP_CODE = :p_map_code;

p_map_code: your_map_code


In [31]:
# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)

# Target database connection
target_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)


start_time = time.time()
print(f"Start of Task: at {start_time}")

OVERRIDE_FLAG = True
p_map_code = "OTC_ORDER_ENTRY_UI"

# Create cursors for both connections
source_cursor = source_conn.cursor()
target_cursor = target_conn.cursor()

# Execute your SQL query to check if the MAP_CODE already exists
target_cursor.execute(f"SELECT MAP_DEFINITION_ID, MAP_CODE FROM {target_table} WHERE MAP_CODE = :p_map_code", {"p_map_code": p_map_code})

# Fetch the results
result = target_cursor.fetchone()
old_map_def_id = result[0]

if result:
    existing_map_code = result[1]
    print(f"MAP_CODE already exists with Existing MAP_CODE: {existing_map_code}")
    
    if OVERRIDE_FLAG:  # Check if OVERRIDE_FLAG is True
        # Delete existing records with the same MAP_CODE
        delete_query = f"DELETE FROM {target_table} WHERE MAP_CODE = '{p_map_code}'"
        print("Delete Query:")
        print(delete_query)  # Print the delete query with dynamic values
        target_cursor.execute(delete_query)
        target_conn.commit()
        
        # Insert the new records
        print("Insert Query:")
        print(insert_def)  # Print the insert query with dynamic values
        insert_print = target_cursor.execute(insert_def, values)
        print(insert_print)
        target_conn.commit()
        print(f"Deleted existing records and inserted new data for MAP_CODE: {p_map_code}")
        print(f"Inserted a new row with MAP_DEFINITION_ID: {next_val}")
        print('pn_map_def_id before calling:', old_map_def_id)
        val_map_insert(vm_df, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST", target_cursor, target_conn, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S", next_val, OVERRIDE_FLAG, old_map_def_id)

else:
    # Execute the INSERT statement with the values dictionary
    print("Insert Query:")
    print(insert_def)  # Print the insert query with dynamic values
    target_cursor.execute(insert_def, values)
    target_conn.commit()  # Commit the transaction
    print("INSERT statement executed successfully.")
    print(f"Inserted a new row with MAP_DEFINITION_ID: {next_val}")
    
    # Call the val_map_insert function for value map data
    val_map_insert(vm_df, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST", target_cursor, target_conn, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S", next_val, OVERRIDE_FLAG, old_map_def_id)


Start of Task: at 1697448784.2665596
MAP_CODE already exists with Existing MAP_CODE: OTC_ORDER_ENTRY_UI
Delete Query:
DELETE FROM OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST WHERE MAP_CODE = 'OTC_ORDER_ENTRY_UI'
Insert Query:

    INSERT INTO OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST (
        MAP_DEFINITION_ID, MAP_CODE, MAP_DESC, ACTIVE_DATE_FLAG, N_VALUE1_SEQUENCE, N_VALUE1_PROMPT_TEXT, N_VALUE1_REQUIRED_FLAG, N_VALUE1_COLUMN_WIDTH, C_VALUE1_SEQUENCE, C_VALUE1_PROMPT_TEXT, C_VALUE1_REQUIRED_FLAG, C_VALUE1_COLUMN_WIDTH, D_VALUE1_SEQUENCE, D_VALUE1_PROMPT_TEXT, D_VALUE1_REQUIRED_FLAG, D_VALUE1_COLUMN_WIDTH, N_VALUE2_SEQUENCE, N_VALUE2_PROMPT_TEXT, N_VALUE2_REQUIRED_FLAG, N_VALUE2_COLUMN_WIDTH, C_VALUE2_SEQUENCE, C_VALUE2_PROMPT_TEXT, C_VALUE2_REQUIRED_FLAG, C_VALUE2_COLUMN_WIDTH, D_VALUE2_SEQUENCE, D_VALUE2_PROMPT_TEXT, D_VALUE2_REQUIRED_FLAG, D_VALUE2_COLUMN_WIDTH, N_VALUE3_SEQUENCE, N_VALUE3_PROMPT_TEXT, N_VALUE3_REQUIRED_FLAG, N_VALUE3_COLUMN_WIDTH, C_VALUE3_SEQUENCE, C_VALUE3_PROMPT_TEXT, C

None
Deleted existing records and inserted new data for MAP_CODE: OTC_ORDER_ENTRY_UI
Inserted a new row with MAP_DEFINITION_ID: 185
pn_map_def_id before calling: 185
Deleted existing records for MAP_DEFINITION_ID: 185
Row 0 inserted successfully.
Deleted existing records for MAP_DEFINITION_ID: 185
Row 1 inserted successfully.
Deleted existing records for MAP_DEFINITION_ID: 185
Row 2 inserted successfully.
Deleted existing records for MAP_DEFINITION_ID: 185
Row 3 inserted successfully.


In [36]:
try:
    # ...
    if OVERRIDE_FLAG:
        # ...
        # Insert the new records
#         target_cursor.execute(insert_def, insert_values)
        target_conn.commit()
        # Print the actual values
        print("Insert Values:")
        for key, value in insert_values.items():
            print(f"{key}: {value}")
        # ...
    else:
        # Execute the INSERT statement with the actual values
#         target_cursor.execute(insert_def, insert_values)
        target_conn.commit()
        # Print the actual values
        print("Insert Values:")
        for key, value in insert_values.items():
            print(f"{key}: {value}")
        # ...
except cx_Oracle.Error as error:
    print(f"Error executing DEF INSERT statement: {error}")


Insert Values:
MAP_DEFINITION_ID: 185
MAP_DESC: OTC Order Entry UI List of Values
ACTIVE_DATE_FLAG: Y
N_VALUE1_SEQUENCE: None
N_VALUE1_PROMPT_TEXT: None
N_VALUE1_REQUIRED_FLAG: None
N_VALUE1_COLUMN_WIDTH: None
C_VALUE1_SEQUENCE: None
C_VALUE1_PROMPT_TEXT: Division
C_VALUE1_REQUIRED_FLAG: Y
C_VALUE1_COLUMN_WIDTH: None
D_VALUE1_SEQUENCE: None
D_VALUE1_PROMPT_TEXT: None
D_VALUE1_REQUIRED_FLAG: None
D_VALUE1_COLUMN_WIDTH: None
N_VALUE2_SEQUENCE: None
N_VALUE2_PROMPT_TEXT: None
N_VALUE2_REQUIRED_FLAG: None
N_VALUE2_COLUMN_WIDTH: None
C_VALUE2_SEQUENCE: None
C_VALUE2_PROMPT_TEXT: Force Extract
C_VALUE2_REQUIRED_FLAG: None
C_VALUE2_COLUMN_WIDTH: None
D_VALUE2_SEQUENCE: None
D_VALUE2_PROMPT_TEXT: None
D_VALUE2_REQUIRED_FLAG: None
D_VALUE2_COLUMN_WIDTH: None
N_VALUE3_SEQUENCE: None
N_VALUE3_PROMPT_TEXT: None
N_VALUE3_REQUIRED_FLAG: None
N_VALUE3_COLUMN_WIDTH: None
C_VALUE3_SEQUENCE: None
C_VALUE3_PROMPT_TEXT: Delivery Type
C_VALUE3_REQUIRED_FLAG: None
C_VALUE3_COLUMN_WIDTH: None
D_VALUE3_SEQUEN

In [37]:
# Assuming you have defined the insert_def and insert_values as before

# Construct the INSERT script as a string
insert_script = insert_def.strip().replace(':MAP_DEFINITION_ID', str(insert_values['MAP_DEFINITION_ID']))

for col in insert_values:
    if col != 'MAP_DEFINITION_ID':
        insert_script = insert_script.replace(f':{col}', str(insert_values[col]))

# Print the INSERT script with actual values
print("Insert Query with Actual Values:")
print(insert_script)


Insert Query with Actual Values:
INSERT INTO OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST (
        MAP_DEFINITION_ID, MAP_DESC, ACTIVE_DATE_FLAG, N_VALUE1_SEQUENCE, N_VALUE1_PROMPT_TEXT, N_VALUE1_REQUIRED_FLAG, N_VALUE1_COLUMN_WIDTH, C_VALUE1_SEQUENCE, C_VALUE1_PROMPT_TEXT, C_VALUE1_REQUIRED_FLAG, C_VALUE1_COLUMN_WIDTH, D_VALUE1_SEQUENCE, D_VALUE1_PROMPT_TEXT, D_VALUE1_REQUIRED_FLAG, D_VALUE1_COLUMN_WIDTH, N_VALUE2_SEQUENCE, N_VALUE2_PROMPT_TEXT, N_VALUE2_REQUIRED_FLAG, N_VALUE2_COLUMN_WIDTH, C_VALUE2_SEQUENCE, C_VALUE2_PROMPT_TEXT, C_VALUE2_REQUIRED_FLAG, C_VALUE2_COLUMN_WIDTH, D_VALUE2_SEQUENCE, D_VALUE2_PROMPT_TEXT, D_VALUE2_REQUIRED_FLAG, D_VALUE2_COLUMN_WIDTH, N_VALUE3_SEQUENCE, N_VALUE3_PROMPT_TEXT, N_VALUE3_REQUIRED_FLAG, N_VALUE3_COLUMN_WIDTH, C_VALUE3_SEQUENCE, C_VALUE3_PROMPT_TEXT, C_VALUE3_REQUIRED_FLAG, C_VALUE3_COLUMN_WIDTH, D_VALUE3_SEQUENCE, D_VALUE3_PROMPT_TEXT, D_VALUE3_REQUIRED_FLAG, D_VALUE3_COLUMN_WIDTH, N_VALUE4_SEQUENCE, N_VALUE4_PROMPT_TEXT, N_VALUE4_REQUIRED_FLAG, N_V

In [39]:
# Assuming you have defined the insert_def and insert_values as before
import datetime
# Function to format values for the INSERT script
def format_value(value):
    if value is None:
        return 'NULL'
    elif isinstance(value, str):
        return f"'{value}'"
    elif isinstance(value, datetime.date):
        formatted_date = value.strftime("%Y-%m-%d %H:%M:%S")
        return f"TO_DATE('{formatted_date}', 'YYYY-MM-DD HH24:MI:SS')"
    elif isinstance(value, (int, float)):
        return str(value)  # No single quotes for numeric values
    else:
        return str(value)

# Construct the INSERT script as a string with formatted values
insert_script = insert_def.strip().replace(':MAP_DEFINITION_ID', str(insert_values['MAP_DEFINITION_ID']))

for col in insert_values:
    if col != 'MAP_DEFINITION_ID':
        value = insert_values[col]
        formatted_value = format_value(value)
        insert_script = insert_script.replace(f':{col}', formatted_value)

# Print the INSERT script with actual values
print("Insert Query with Actual Values:")
print(insert_script)


Insert Query with Actual Values:
INSERT INTO OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST (
        MAP_DEFINITION_ID, MAP_DESC, ACTIVE_DATE_FLAG, N_VALUE1_SEQUENCE, N_VALUE1_PROMPT_TEXT, N_VALUE1_REQUIRED_FLAG, N_VALUE1_COLUMN_WIDTH, C_VALUE1_SEQUENCE, C_VALUE1_PROMPT_TEXT, C_VALUE1_REQUIRED_FLAG, C_VALUE1_COLUMN_WIDTH, D_VALUE1_SEQUENCE, D_VALUE1_PROMPT_TEXT, D_VALUE1_REQUIRED_FLAG, D_VALUE1_COLUMN_WIDTH, N_VALUE2_SEQUENCE, N_VALUE2_PROMPT_TEXT, N_VALUE2_REQUIRED_FLAG, N_VALUE2_COLUMN_WIDTH, C_VALUE2_SEQUENCE, C_VALUE2_PROMPT_TEXT, C_VALUE2_REQUIRED_FLAG, C_VALUE2_COLUMN_WIDTH, D_VALUE2_SEQUENCE, D_VALUE2_PROMPT_TEXT, D_VALUE2_REQUIRED_FLAG, D_VALUE2_COLUMN_WIDTH, N_VALUE3_SEQUENCE, N_VALUE3_PROMPT_TEXT, N_VALUE3_REQUIRED_FLAG, N_VALUE3_COLUMN_WIDTH, C_VALUE3_SEQUENCE, C_VALUE3_PROMPT_TEXT, C_VALUE3_REQUIRED_FLAG, C_VALUE3_COLUMN_WIDTH, D_VALUE3_SEQUENCE, D_VALUE3_PROMPT_TEXT, D_VALUE3_REQUIRED_FLAG, D_VALUE3_COLUMN_WIDTH, N_VALUE4_SEQUENCE, N_VALUE4_PROMPT_TEXT, N_VALUE4_REQUIRED_FLAG, N_V